### Introduction to Data Ingestion

In [7]:
import langchain
from typing import List, Dict, Any
import pandas as pd

In [9]:
from langchain_core.documents import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print("Setup completed. Ready to start data ingestion!")

Setup completed. Ready to start data ingestion!


### Understanding Document Structure in Langchain

In [14]:
## Create a simple document
doc = Document(
    page_content="This is the main text content that will be embedded and searched",
    metadata={
        "source": "example.txt",
        "page": 1,
        "date_created":"2024-01-01",
        "custom_field":"any_value"
    }
)

print(doc)

print(f"Content: {doc.page_content}")
print(f"Content: {doc.metadata['source']}")

# Why Metadata Matters?
print("\n Metadata is crucia for:")
print("- Filtering search results")
print("- Tracking document sources")
print("- Providing context in responses")
print("- Debugging and auditing")

page_content='This is the main text content that will be embedded and searched' metadata={'source': 'example.txt', 'page': 1, 'date_created': '2024-01-01', 'custom_field': 'any_value'}
Content: This is the main text content that will be embedded and searched
Content: example.txt

 Metadata is crucia for:
- Filtering search results
- Tracking document sources
- Providing context in responses
- Debugging and auditing


### Text Files (*.txt) - The Simplest Case

In [15]:
## Create a simple text file
import os
os.makedirs("data/text_files", exist_ok=True)

In [18]:
sample_texts= {
    "data/text_files/python_intro.txt": """
    Python is a high-level, interpreted programming language known for its simplicity and readability. 
    It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, 
    and functional programming.
    
    One of the key features of Python is its extensive standard library, which provides ready-to-use modules for a wide range of tasks, from file I/O to web development. 
    This makes it an excellent choice for both beginners and experienced developers.
    
    Python's syntax is designed to be intuitive and easy to learn, making it accessible even to those with no prior programming experience. 
    Its dynamic typing system allows for more flexible code writing, while its garbage collection mechanism simplifies memory management.
    Popular applications of Python include web development (using frameworks like Django and Flask), data analysis (with libraries such as Pandas and NumPy), 
    artificial intelligence (through TensorFlow and PyTorch), and automation scripts. The language's versatility has led to its widespread adoption in various industries, 
    from finance to healthcare.
"""
}

for filepath, content in sample_texts.items():
    with open(filepath, "w", encoding="utf-8") as file:
        file.write(content)
        
print("Sample files created successfully.")
        

Sample files created successfully.


### TextLoader - Read Single File

In [26]:
from langchain.document_loaders import TextLoader

## Loading a single text file
loader = TextLoader(
    "data/text_files/python_intro.txt",
    encoding='utf-8'
)

documents = loader.load()
print(f"Loaded {len(documents)}")
print(f"Document content: {documents[0].page_content[:100]}...")
print(f"Metadata: {documents[0].metadata}")

Loaded 1
Document content: 
    Python is a high-level, interpreted programming language known for its simplicity and readabili...
Metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader - Multiple text Files

In [30]:
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # Pattern to match files
    loader_cls=TextLoader, # Loader class to use
    loader_kwargs={"encoding": 'utf-8'},
    show_progress=True
)

documents = dir_loader.load()

print(f"Loaded {len(documents)}")
for i, doc in enumerate(documents):
    print(f"\nDocuments {i+1}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Length: {len(doc.page_content)} characters")
    

# Analysis
print("\nDirectoryLoader Characteristics\n")
print("Advantages:")
print("- Loads multiple files at onece")
print("- supports global patterns")
print("- progress tracking")
print("- Recursive directory scanning")

print("\nDisadvantages:")
print("- all files must be same type")
print("- limited error handling per file")
print("- can be memory intensive for large directories or large files")

100%|██████████| 1/1 [00:00<00:00, 1882.54it/s]

Loaded 1

Documents 1
Source: data/text_files/python_intro.txt
Length: 1192 characters

DirectoryLoader Characteristics

Advantages:
- Loads multiple files at onece
- supports global patterns
- progress tracking
- Recursive directory scanning

Disadvantages:
- all files must be same type
- limited error handling per file
- can be memory intensive for large directories or large files


## Text Splitting Strategies

In [31]:
### Different text splitting strategies
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content="\n    Python is a high-level, interpreted programming language known for its simplicity and readability. \n    It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, \n    and functional programming.\n\n    One of the key features of Python is its extensive standard library, which provides ready-to-use modules for a wide range of tasks, from file I/O to web development. \n    This makes it an excellent choice for both beginners and experienced developers.\n\n    Python's syntax is designed to be intuitive and easy to learn, making it accessible even to those with no prior programming experience. \n    Its dynamic typing system allows for more flexible code writing, while its garbage collection mechanism simplifies memory management.\n    Popular applications of Python include web development (using frameworks lik

In [46]:
### Method 1 - Character Text Splitter
text = documents[0].page_content
text

"\n    Python is a high-level, interpreted programming language known for its simplicity and readability. \n    It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, \n    and functional programming.\n\n    One of the key features of Python is its extensive standard library, which provides ready-to-use modules for a wide range of tasks, from file I/O to web development. \n    This makes it an excellent choice for both beginners and experienced developers.\n\n    Python's syntax is designed to be intuitive and easy to learn, making it accessible even to those with no prior programming experience. \n    Its dynamic typing system allows for more flexible code writing, while its garbage collection mechanism simplifies memory management.\n    Popular applications of Python include web development (using frameworks like Django and Flask), data analysis (with libraries such as Pandas and NumPy), \n

In [47]:
print("CharacterTextSplitter")
splitter = CharacterTextSplitter(
    separator="\n", # Split on new lines
    chunk_size=200, # Max chunk size in characters
    chunk_overlap=20, # Overlap between chunks
    length_function=len # how to measure the length of a text chunk
)

char_chunks = splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}...")

CharacterTextSplitter
Created 9 chunks
First chunk: Python is a high-level, interpreted programming language known for its simplicity and readability....


In [48]:
print(char_chunks[0])
print("-------------")
print(char_chunks[1])
print("-------------")
print(char_chunks[2])
print("-------------")

Python is a high-level, interpreted programming language known for its simplicity and readability.
-------------
It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, 
    and functional programming.
-------------
One of the key features of Python is its extensive standard library, which provides ready-to-use modules for a wide range of tasks, from file I/O to web development.
-------------


In [49]:
# Method 2 - Recursive Character Text Splitter
# Most recommended text splitter technique
print("\nRecursiveCharacterTextSplitter")
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ","], # Try this separators in order
    chunk_size=200,
    chunk_overlap=20,
    length_function=len
)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}...")


RecursiveCharacterTextSplitter
Created 9 chunks
First chunk: Python is a high-level, interpreted programming language known for its simplicity and readability....


In [50]:
print(recursive_chunks[0])
print("-------------")
print(recursive_chunks[1])
print("-------------")

Python is a high-level, interpreted programming language known for its simplicity and readability.
-------------
It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, 
    and functional programming.
-------------


In [53]:
# Method 3: TokenTextSplitter
print("\nTokenTextSplitter")
token_splitter = TokenTextSplitter(
    chunk_size=50, # Size in tokens (not characters)
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(text)
print(f"Created {len(token_chunks)} chunks")
print(f"First chunk: {token_chunks[0][:100]}...")

print(token_chunks[0])
print("-------------")
print(token_chunks[1])
print("-------------")


TokenTextSplitter
Created 7 chunks
First chunk: 
    Python is a high-level, interpreted programming language known for its simplicity and readabili...

    Python is a high-level, interpreted programming language known for its simplicity and readability. 
    It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms,
-------------
 1991. Python supports multiple programming paradigms, including procedural, object-oriented, 
    and functional programming.

    One of the key features of Python is its extensive standard library, which provides ready-to-use
-------------


## Comparison

#### CharacterTextSplitter
* Simple and Predictable
* Good for structured text
* May break mid-sentence  
**Use when:** text has clear delimiters

#### RecursiveCharacterTextSplitter
* Respects text structure
* Tries multiple seperators
* best general-purpose splitter
* Slightly more complex  
**Use when:** Default choice for most texts

#### TokentextSplitter
* Respects model token limits
* More accurate for embeddings
* Slower than character-based  
**Use When:** Working with token-limited models